In [ ]:
# class binary_controler:
#     def __init__(self, bin_location, output_location, output_name):
#         self.bin_location = bin_location
#         self.output_location = output_location
#         self.output_name = output_name
#         self.cfglist = []
        
#     def handle_CFG(self):
#         self.project = angr.Project(self.bin_location, load_options={'auto_load_libs': False})
#         self.cfg = self.project.analyses.CFGFast()
        
#     def derivate_fun_cfg(self):
#         i = 0
#         for function_addr, function in self.cfg.kb.functions.items():
#             func_cfg = self.project.analyses.CFGFast(force_complete_scan=False, start_at_entry=function_addr, function_prologues=True)
#             self.cfglist.append(func_cfg)
#             print(function_addr,function)
#             i = i + 1
#             if (i > 4):
#                 break

In [ ]:
import angr
import networkx as nx
from angrutils import *

In [ ]:
# bin_file = binary_controler('/usr/local/binutils/bin/addr2line', '~/PycharmProjects/prakProject/output_CFG', "CFG")

In [ ]:
# bin_file.handle_CFG()

In [ ]:
# bin_file.derivate_fun_cfg()

In [ ]:
# for cfg in bin_file.cfglist:
#     plot_cfg(cfg, "ais3_cfg", asminst=True, remove_imports=True, remove_path_terminator=True) 

In [ ]:
# project = angr.Project('/usr/local/binutils/bin/addr2line', load_options={'auto_load_libs': False})
# cfg = project.analyses.CFGFast()
# for func_addr in cfg.kb.functions:
#     func = cfg.kb.functions[func_addr]
#     print(func.name)

In [ ]:
# for func_addr in cfg.kb.functions:
#     func = cfg.kb.functions[func_addr]
#     if (func.name == "mkstemps")
#     print(func.name)

In [ ]:
# project = angr.Project('/usr/local/binutils/bin/addr2line', load_options={'auto_load_libs': False})
# cfg = project.analyses.CFGFast()
# function_cfgs = {}

# for func_addr in cfg.kb.functions:
#     func = cfg.kb.functions[func_addr]
#     ## 添加对function用法的判定，备注只有相同的元文件具有相同函数名的函数才能才一起进行测试
#     ## 如果node的数量小于3,则不对该函数进行添加
#     length = 0
#     for _ in func.blocks:
#         length = length +1
#     if (func.name.startswith('sub_') or length <= 3):
#         continue
        
#     G = nx.DiGraph()
    
#     blocks = list(func.blocks)
    
#     for block in blocks:
#         G.add_node(block.addr, block = block)

#     for block in blocks:
#         command = block.disassembly
#         instr = command.insns[-1]
#         try :
#             address = int(instr.op_str,16)
#         except ValueError:
#             continue
#         print(len(successors))
#         for succ in successors:
#             if succ.addr in G:
#                 G.add_edge(block.addr, succ.addr)
                
#     function_cfgs[func.name] = G

In [ ]:
project = angr.Project('/usr/local/binutils/bin/addr2line', load_options={'auto_load_libs': False})
cfg = project.analyses.CFGFast()
function_cfgs = {}

In [ ]:
for func_addr in cfg.kb.functions:
    func = cfg.kb.functions[func_addr]
    ## 添加对function用法的判定，备注只有相同的元文件具有相同函数名的函数才能才一起进行测试
    ## 如果node的数量小于3,则不对该函数进行添加
    length = 0
#     for node in func.nodes:
#         print(type(node))
#     print(type(func))
    for _ in func.blocks:
        length = length +1
    if (func.name.startswith('sub_') or length <= 3 or func.is_syscall):
        continue
        
    G = nx.DiGraph()
    
    blocks = list(func.blocks)
    
    for block in blocks:
        G.add_node(block.addr, block = block)
#         print(type(block.addr))

    for block in blocks:
        node = cfg.get_any_node(block.addr)
        successors = cfg.get_successors(node)
#         print(len(successors))
        for succ in successors:
            if succ.addr in G:
                G.add_edge(block.addr, succ.addr)
                
    function_cfgs[func.name] = G

In [ ]:
# data_sections = [section for section in project.loader.main_object.sections if section.is_readable and section.name in ['.data', '.rodata']]

In [ ]:
# print(type(data_sections[0]))

In [ ]:
# node = cfg.get_any_node(int("0x433730",16))
# print(type(node))
# print(node.successors)
# for i in node.successors:
#     print(hex(i.addr))
#     print(type(i.addr))  ## test node

In [ ]:
# for name in function_cfgs:
#     print(name)  ##show function node

In [ ]:
# i = 1
# for name in function_cfgs:
#     i = i + 1
#     print(function_cfgs[name].number_of_nodes())
#     if(i > 30):
#         break

In [ ]:
def count_string_constants(block):
    constants = 0
#     print(type(block))
    for insn in block.capstone.insns:
#         print(type(insn))
        for operand in insn.operands:
#             print(operand.value)
#            print(type(operand))
            if operand.type == 2:  # memory.data
                mem_addr = operand.mem.disp
                try:
                    # 尝试将值解释为字符串
                    string = project.loader.memory.load(mem_addr, 20).tobytes()
                    if is_printable_string(string):
                        constants = constants + 1
                except:
                    continue
    return constants


In [ ]:
def count_num_constants(block):
    numeric_constants = 0
    for insn in block.capstone.insns:
        for operand in insn.operands:
            if operand.type == 1:  # Immediate value
                numeric_constants += 1
    return numeric_constants


In [ ]:
def count_transfer(block):
    transfer_commands = {'jmp', 'je', 'jne', 'jg', 'jge', 'jl', 'jle','ja', 'jae', 'jb', 'jbe',
                        'loop', 'loope', 'loopne','jo', 'jno', 'js', 'jns','jp', 'jnp'}
    transfers = 0
    for insn in block.capstone.insns:
        if insn.mnemonic in transfer_commands:
            transfers+=1
    return transfers

In [ ]:
def count_calls(block):
    calls = 0
    for insn in block.capstone.insns:
        if insn.mnemonic == 'call':
            calls+=1
    return calls

In [ ]:
def count_instructions(block):
    return block.size

In [ ]:
def count_arithmetic(block):
    arithmetic_commands = { ## only include the normalest command
        'add', 'sub', 'mul', 'imul', 'div', 'idiv','inc', 'dec', 'neg', 'adc', 'sbb'
    }
    arithmetics = 0
    for insn in block.capstone.insns:
        if insn.mnemonic in arithmetic_commands:
            arithmetics += 1
    
    return arithmetics

In [ ]:
def count_offspring(block_addr, cfg_graph):
    visited_node = set()
    offsprings = 0
    stack = [block_addr]
    while stack:
        position = stack.pop()
        if position not in visited_node:
            visited_node.add(position)
            for successor in cfg_graph.successors(position):
                if successor not in visited_node:
                    stack.append(successor)
                    offsprings += 1
    return offsprings
    

In [ ]:
def refin(acfg_graph):
    new_acfg = nx.DiGraph()
    for block_addr in acfg_graph.nodes:
        new_acfg.add_node(tuple(acfg_graph.nodes[block_addr]['block_attr']))
        
    for block_addr_1,block_addr_2 in acfg_graph.edges:
        new_acfg.add_edge(tuple(acfg_graph.nodes[block_addr_1]['block_attr']),tuple(acfg_graph.nodes[block_addr_2]['block_attr']))
    return new_acfg

In [ ]:
## 将提取出的CFG变成ACFG
## 依据论文提取的block attribution
## 
def cfg_to_acfg(cfg_graph):
    acfg_graph = nx.DiGraph()
    for block_addr in cfg_graph.nodes:
#         print(type(block))
        block = cfg_graph.nodes[block_addr]['block']
        block_attr = []
        block_attr.append(count_string_constants(block))
        block_attr.append(count_num_constants(block))
        block_attr.append(count_transfer(block))
        block_attr.append(count_calls(block))
        block_attr.append(count_instructions(block))
        block_attr.append(count_arithmetic(block))
        block_attr.append(count_offspring(block_addr, cfg_graph))
        acfg_graph.add_node(block_addr, block_attr = block_attr)
    for u,v in cfg_graph.edges:
        acfg_graph.add_edge(u,v)
    blocks_between = nx.betweenness_centrality(cfg_graph)
    for block_addr, centrality in blocks_between.items():
        acfg_graph.nodes[block_addr]['block_attr'].append(centrality)
#         print(acfg_graph.nodes[block_addr]['block_attr'])
        
    refin_acg = refin(acfg_graph)
#     print(refin_acg)
    return refin_acg
    
    
    

In [ ]:
graph_list = []
name_list = []

for name in function_cfgs:
    cfg_graph = function_cfgs[name]
#     print(type(list(cfg_graph.nodes)[0]))
    acfg_graph = cfg_to_acfg(cfg_graph)  ##将cfg变成acfg的运用
    graph_list.append(acfg_graph)
    name_list.append(name)


In [ ]:
for graph in graph_list:
    print(type(graph[1]))
    print(type(graph[0])) 
    print(type(graph[2]))

In [ ]:
import pickle

with open("./graphs.pkl", "wb") as f:
    pickle.dump(graph_list, f)
    
with open("./names.pkl","wb") as f:
    pickle.dump(name_list, f)

In [ ]:
print(name_list)

In [ ]:
print(graph_list)

In [ ]:
## 对每个ACFG标记funcion name 和源文件名

In [ ]:
# for name in function_cfgs:
#     used_cfg = function_cfgs[name]
#     for block_addr in used_cfg:
#         block.string = usedc
    

In [ ]:
# project = angr.Project('./stringtest2', load_options={'auto_load_libs': False})
# cfg = project.analyses.CFGFast()
# function_cfgs = {}

In [ ]:
# for func_addr, func in cfg_2.kb.functions.items():
#     if(func.name == 'main'):
#         print(f"Function at {hex(func_addr)}:")
#         # 遍历基本块
#         for block in func.blocks:
#         # 反汇编基本块
#             block.pp()

In [ ]:
# print(type(cfg))